In [1]:
import os
from types import SimpleNamespace   

from kornia.utils.grid import create_meshgrid3d
import torch
import numpy as np

from models.networks import NGPGv2
from models.rendering_NGPA import render
from utils.utils import slim_ckpt, load_ckpt
from datasets import dataset_dict
from datasets.NGPA.colmap import name_to_task, ColmapDataset_NGPA
from datasets.colmap_utils import read_cameras_binary, read_images_binary

In [2]:
hparams = {
    "root_dir": "dataset/WAT/breville",
    "dataset_name": "colmap_ngpa_render",
    "exp_name": "breville",
    "downsample": 1.0,
    "num_epochs": 20,
    "batch_size": 8192,
    "lr": 1e-2,
    "eval_lpips": True,
    "task_curr": 4,
    "task_number": 5,
    "dim_a": 48,
    "dim_g": 16,
    "scale": 8.0,
    "vocab_size": 5,
    "weight_path": "ckpts/NGPGv2/colmap_ngpa/breville/epoch=19-v6.ckpt",
    "render_fname": "UB",
    "val_only": True,
    "use_exposure": False,
}
hparams = SimpleNamespace(**hparams)

### Setup

In [4]:
dataset = dataset_dict[hparams.dataset_name]
kwargs = {'root_dir': hparams.root_dir,
        'downsample': hparams.downsample}
test_dataset = dataset(split='test', **kwargs)

self.img_wh = (1920, 1440)
[test] near_far = 0.2990965247154236/60.96563720703125, scale = 7.620704650878906
Loading 34 test images ...


100%|██████████| 34/34 [00:02<00:00, 12.23it/s]


self.poses_interpolate = torch.Size([2934, 3, 4]), self.ts_interpolate = torch.Size([2934]), self.task_ids_interpolate = 2934


In [ ]:
test_dataset[0]

In [ ]:
rgb_act = 'None' if hparams.use_exposure else 'Sigmoid'
model = NGPGv2(scale=hparams.scale, vocab_size=hparams.task_curr+1, rgb_act=rgb_act, dim_a=hparams.dim_a, dim_g=hparams.dim_g)
G = model.grid_size
model.register_buffer('density_grid', torch.zeros(model.cascades, G**3))
model.register_buffer('grid_coords', create_meshgrid3d(G, G, G, False, dtype=torch.int32).reshape(-1, 3))

In [ ]:
load_ckpt(model, hparams.weight_path)

In [ ]:
model

In [ ]:
list(model.named_buffers())

In [ ]:
test_dataset.directions

### Check data split paths

In [34]:
data_dir = "dataset/WAT/breville"
folder = "images"

In [35]:
imdata = read_images_binary(os.path.join(data_dir, 'sparse/0/images.bin'))
img_names = [imdata[k].name for k in imdata]
img_names

['IMG_9184/IMG_9184_100.png',
 'IMG_9184/IMG_9184_110.png',
 'IMG_9184/IMG_9184_0.png',
 'IMG_9184/IMG_9184_170.png',
 'IMG_9184/IMG_9184_130.png',
 'IMG_9184/IMG_9184_160.png',
 'IMG_9184/IMG_9184_120.png',
 'IMG_9184/IMG_9184_240.png',
 'IMG_9184/IMG_9184_10.png',
 'IMG_9184/IMG_9184_230.png',
 'IMG_9184/IMG_9184_150.png',
 'IMG_9184/IMG_9184_220.png',
 'IMG_9184/IMG_9184_250.png',
 'IMG_9184/IMG_9184_270.png',
 'IMG_9184/IMG_9184_140.png',
 'IMG_9184/IMG_9184_190.png',
 'IMG_9184/IMG_9184_210.png',
 'IMG_9184/IMG_9184_280.png',
 'IMG_9184/IMG_9184_180.png',
 'IMG_9184/IMG_9184_200.png',
 'IMG_9184/IMG_9184_300.png',
 'IMG_9184/IMG_9184_310.png',
 'IMG_9184/IMG_9184_20.png',
 'IMG_9184/IMG_9184_260.png',
 'IMG_9184/IMG_9184_320.png',
 'IMG_9184/IMG_9184_290.png',
 'IMG_9184/IMG_9184_360.png',
 'IMG_9184/IMG_9184_380.png',
 'IMG_9184/IMG_9184_370.png',
 'IMG_9184/IMG_9184_420.png',
 'IMG_9184/IMG_9184_410.png',
 'IMG_9185/IMG_9185_120.png',
 'IMG_9184/IMG_9184_340.png',
 'IMG_9184/IMG

In [36]:
img_paths = [os.path.join(data_dir, folder, name) for name in sorted(img_names)]
img_paths

['dataset/WAT/breville/images/IMG_9184/IMG_9184_0.png',
 'dataset/WAT/breville/images/IMG_9184/IMG_9184_10.png',
 'dataset/WAT/breville/images/IMG_9184/IMG_9184_100.png',
 'dataset/WAT/breville/images/IMG_9184/IMG_9184_110.png',
 'dataset/WAT/breville/images/IMG_9184/IMG_9184_120.png',
 'dataset/WAT/breville/images/IMG_9184/IMG_9184_130.png',
 'dataset/WAT/breville/images/IMG_9184/IMG_9184_140.png',
 'dataset/WAT/breville/images/IMG_9184/IMG_9184_150.png',
 'dataset/WAT/breville/images/IMG_9184/IMG_9184_160.png',
 'dataset/WAT/breville/images/IMG_9184/IMG_9184_170.png',
 'dataset/WAT/breville/images/IMG_9184/IMG_9184_180.png',
 'dataset/WAT/breville/images/IMG_9184/IMG_9184_190.png',
 'dataset/WAT/breville/images/IMG_9184/IMG_9184_20.png',
 'dataset/WAT/breville/images/IMG_9184/IMG_9184_200.png',
 'dataset/WAT/breville/images/IMG_9184/IMG_9184_210.png',
 'dataset/WAT/breville/images/IMG_9184/IMG_9184_220.png',
 'dataset/WAT/breville/images/IMG_9184/IMG_9184_230.png',
 'dataset/WAT/brev

In [37]:
cam_infos = sorted(img_paths, key = lambda x : x.split('/')[-1])
cam_infos = sorted(img_paths)
test_cam_infos = [c for idx, c in enumerate(cam_infos) if idx % 8 == 0]
test_cam_infos


['dataset/WAT/breville/images/IMG_9184/IMG_9184_0.png',
 'dataset/WAT/breville/images/IMG_9184/IMG_9184_160.png',
 'dataset/WAT/breville/images/IMG_9184/IMG_9184_230.png',
 'dataset/WAT/breville/images/IMG_9184/IMG_9184_300.png',
 'dataset/WAT/breville/images/IMG_9184/IMG_9184_380.png',
 'dataset/WAT/breville/images/IMG_9184/IMG_9184_450.png',
 'dataset/WAT/breville/images/IMG_9184/IMG_9184_520.png',
 'dataset/WAT/breville/images/IMG_9185/IMG_9185_0.png',
 'dataset/WAT/breville/images/IMG_9185/IMG_9185_160.png',
 'dataset/WAT/breville/images/IMG_9185/IMG_9185_230.png',
 'dataset/WAT/breville/images/IMG_9185/IMG_9185_300.png',
 'dataset/WAT/breville/images/IMG_9185/IMG_9185_380.png',
 'dataset/WAT/breville/images/IMG_9185/IMG_9185_450.png',
 'dataset/WAT/breville/images/IMG_9185/IMG_9185_520.png',
 'dataset/WAT/breville/images/IMG_9185/IMG_9185_60.png',
 'dataset/WAT/breville/images/IMG_9186/IMG_9186_110.png',
 'dataset/WAT/breville/images/IMG_9186/IMG_9186_190.png',
 'dataset/WAT/brevi

In [39]:
len(test_cam_infos)

34

In [30]:
task_ids, test_img_ids = name_to_task(img_paths)
test_img_ids

[0,
 8,
 16,
 24,
 32,
 40,
 48,
 56,
 64,
 72,
 80,
 84,
 92,
 100,
 108,
 116,
 124,
 132,
 140,
 148,
 156,
 164,
 169,
 177,
 185,
 193,
 201,
 209,
 217,
 225,
 233,
 241,
 244,
 252,
 260,
 268,
 276,
 284,
 292,
 300,
 308,
 316,
 323,
 331,
 339,
 347,
 355,
 363,
 371,
 379,
 387,
 395,
 403]

In [31]:
test_img_paths = [x for i, x in enumerate(img_paths) if i in test_img_ids]
test_img_paths # Same as 3DGS!

['dataset/WAT/car_resized/images/IMG_2285/IMG_2285_0.png',
 'dataset/WAT/car_resized/images/IMG_2285/IMG_2285_1240.png',
 'dataset/WAT/car_resized/images/IMG_2285/IMG_2285_1560.png',
 'dataset/WAT/car_resized/images/IMG_2285/IMG_2285_1840.png',
 'dataset/WAT/car_resized/images/IMG_2285/IMG_2285_2120.png',
 'dataset/WAT/car_resized/images/IMG_2285/IMG_2285_2400.png',
 'dataset/WAT/car_resized/images/IMG_2285/IMG_2285_2720.png',
 'dataset/WAT/car_resized/images/IMG_2285/IMG_2285_3000.png',
 'dataset/WAT/car_resized/images/IMG_2285/IMG_2285_3280.png',
 'dataset/WAT/car_resized/images/IMG_2285/IMG_2285_560.png',
 'dataset/WAT/car_resized/images/IMG_2285/IMG_2285_840.png',
 'dataset/WAT/car_resized/images/IMG_2286/IMG_2286_0.png',
 'dataset/WAT/car_resized/images/IMG_2286/IMG_2286_1240.png',
 'dataset/WAT/car_resized/images/IMG_2286/IMG_2286_1560.png',
 'dataset/WAT/car_resized/images/IMG_2286/IMG_2286_1840.png',
 'dataset/WAT/car_resized/images/IMG_2286/IMG_2286_2120.png',
 'dataset/WAT/ca

In [32]:
len(test_img_paths)

53

In [33]:
test_img_paths

['dataset/WAT/car_resized/images/IMG_2285/IMG_2285_0.png',
 'dataset/WAT/car_resized/images/IMG_2285/IMG_2285_1240.png',
 'dataset/WAT/car_resized/images/IMG_2285/IMG_2285_1560.png',
 'dataset/WAT/car_resized/images/IMG_2285/IMG_2285_1840.png',
 'dataset/WAT/car_resized/images/IMG_2285/IMG_2285_2120.png',
 'dataset/WAT/car_resized/images/IMG_2285/IMG_2285_2400.png',
 'dataset/WAT/car_resized/images/IMG_2285/IMG_2285_2720.png',
 'dataset/WAT/car_resized/images/IMG_2285/IMG_2285_3000.png',
 'dataset/WAT/car_resized/images/IMG_2285/IMG_2285_3280.png',
 'dataset/WAT/car_resized/images/IMG_2285/IMG_2285_560.png',
 'dataset/WAT/car_resized/images/IMG_2285/IMG_2285_840.png',
 'dataset/WAT/car_resized/images/IMG_2286/IMG_2286_0.png',
 'dataset/WAT/car_resized/images/IMG_2286/IMG_2286_1240.png',
 'dataset/WAT/car_resized/images/IMG_2286/IMG_2286_1560.png',
 'dataset/WAT/car_resized/images/IMG_2286/IMG_2286_1840.png',
 'dataset/WAT/car_resized/images/IMG_2286/IMG_2286_2120.png',
 'dataset/WAT/ca